# Master notebook with other notebook reference and SQL join references

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import operator
from sqlalchemy import create_engine
import psycopg2
import plotly.express as px
import warnings
#import pygraphviz
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
#python -m ipykernel install --user --name graphviz_oss --display-name "graphviz_oss_smn"
import datatable as dt
import sidetable

## Dependency Edgelist Creation
* with license and github homepage only

In [ ]:
meta = pd.read_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/pypi_meta_with_license_github_slugs.csv',low_memory=False) #read data
meta = meta.drop('Unnamed: 0', axis=1) #drop row name column
meta['dependency'] = meta['dependency'].apply(lambda x: 'none available' if x is np.NaN else x)
meta.name = meta.name.apply(lambda x: str(x).lower())

In [ ]:
pd.set_option('display.max_colwidth', None)
meta['one_dep'] = meta['dependency'].apply(lambda x: x.split(' ')[0].split('=')[0].split('.')[0].split('~')[0].split('>')[0].split('[')[0].split('<')[0].replace(';', ''))
dep_df = meta[['name','one_dep']]
dep_df.columns = ['name','dependency_name']
dep_df = dep_df.drop_duplicates()

In [ ]:
dep_df.shape

In [ ]:
dep_df.to_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/dependency_edgelist_03072022.csv')

## Slugs for packages in dependency edgelist

In [ ]:
meta = pd.read_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/pypi_meta_with_license_github_slugs.csv',low_memory=False) #read data
meta = meta.drop('Unnamed: 0', axis=1) #drop row name column
print("Nulls in all columns:",meta.isnull().sum())
print("Uniq no dependency packages: ",meta[meta.dependency.isna()==True].name.nunique())
#meta['dependency'] = meta['dependency'].apply(lambda x: 'none available' if x is np.NaN else x)
meta = meta.apply(lambda x: x.astype(str).str.lower())
meta.head()

In [ ]:
meta[meta.name.str.contains('xstatic-term')]

In [ ]:
meta[meta.slug.str.rstrip('/').str.contains('.*/.*/')]

In [ ]:
meta[meta.slug.str.contains('.*/.*/')]

In [ ]:
df = meta.copy()
df.shape

In [ ]:
df_filtered = df[~df.slug.str.contains('.*github.com/')]
df_filtered.shape

In [ ]:
#create new dataframe with unclean slugs, i.e, filtered on thsoe with github.com in the front
df_unclean = df[df.slug.str.contains('.*github.com/')]

In [ ]:
#replace the github.com with ""
df_unclean.slug = df_unclean.slug.str.replace('.*github.com/', "")

In [ ]:
df_unclean.shape

In [ ]:
#concat cleaned rows with original filtered df
df_clean = pd.concat([df_filtered,df_unclean])
df_clean.shape

In [ ]:
df_clean[df_clean.slug.str[-1]=='/']

In [ ]:
#make a df to now clean slashes at the end of string by using rstrip
df_no_slash = df_clean.copy()
df_no_slash.slug = df_no_slash.slug.str.rstrip('//')

In [ ]:
df_no_slash.shape

In [ ]:
#new df with no github.io and slashes at end
#(df_no_slash.slug.str.contains('/')) |
df_new_slugs = df_no_slash[~( (df_no_slash.slug.str.contains('github.io')) )]

In [ ]:
df_new_slugs.shape

In [ ]:
'''filtering out those with single slugs. This is concatenated with single slug 
(username missing package names) df after fixing it later.
'''
df_no_single_slug = df_new_slugs[df_new_slugs.slug.str.contains('/')]
df_no_single_slug.shape

In [ ]:
'''filter rows with slugs wwhic ahve packagenames but no usernames. 
This is fixed and later concatenated with proper slugs'''
df_single_slugs = df_new_slugs[~df_new_slugs.slug.str.contains('/')]

In [ ]:
df_new_slugs.head()

In [ ]:
#create slugs for only usernames hopin it will be a slug
df_single_slugs.slug = df_single_slugs.slug+'/'+df_single_slugs.name
df_single_slugs.sample(5)

In [ ]:
df_single_slugs.shape

In [ ]:
#concat both df with single slugs fixed and proper slugs
df_full_slugs = pd.concat([df_no_single_slug,df_single_slugs])
df_full_slugs.shape

In [ ]:
df_full_slugs.info()

In [ ]:
df_full_slugs = df_full_slugs[['name','slug']]
df_full_slugs.shape

In [ ]:
df_full_slugs_no_dups = df_full_slugs.drop_duplicates()
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip('"')
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip('{')
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip('}')
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip('~')
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip("'")
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.rstrip("/")
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip("#")
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.slug = df_full_slugs_no_dups.slug.str.strip("*")
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups.shape

In [ ]:
df_full_slugs_no_dups = df_full_slugs_no_dups.drop_duplicates()
df_full_slugs_no_dups.shape

In [ ]:
slug_count = pd.DataFrame(df_full_slugs_no_dups.value_counts()).reset_index()
slug_count.columns=['name','slug','count']
slug_count[slug_count['count']>1]

In [ ]:
df_full_slugs_no_dups[df_full_slugs_no_dups.slug=='smn/django-dirtyfields']

In [ ]:
pd.DataFrame(df_full_slugs_no_dups.groupby('slug').count()).sample(20)

In [ ]:
#Save as csv
df_full_slugs_no_dups.to_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/oss_pypi_2021_cleaner_slugs_04022022.csv')

In [ ]:
#making conenction to postgresSQL DB
pgpassword = os.getenv('password')
pguser = os.getenv('user')
host =  'postgis1-s.bii.virginia.edu'
database = 'sdad'
#port = 

#trying connection
dbserver_p = psycopg2.connect(
    user=pguser, 
    password=pgpassword, 
    host=host,
    database=database 
)

#dbserver_p.autocommit = True
cursor_p = dbserver_p.cursor()
engine_p = create_engine("postgresql+psycopg2://{user}:{pw}@{host}/{db}"
                       .format(user=pguser, pw=pgpassword, host=host, db=database))

In [ ]:
# Take in a PostgreSQL engine and query and return dataframe
def load_db_table(query, eng):
    '''
    Function which will use connection engine with pd.read_sql_query
    '''
    data = pd.read_sql_query(query, con=eng)
    return data

def load_tbl_cur(query, cursor):
    '''
    Function which will use cursor and fetch all the rows
    from result of sql query and convert to pandas dataframe
    '''
    cursor.execute(query)
    df = cursor.fetchall()
    colnames = [x[0] for x in cursor.description]
    return pd.DataFrame(df, columns=colnames)

## Quality check

In [ ]:
#create a table from cleaned slugs csv 04022022
df_full_slugs_no_dups.to_sql('pypi_meta_slug_cleaner_04022022', con = engine_p, \
                     index=False, chunksize=1000, if_exists = 'replace', schema='gh')

In [ ]:
query = '''
SELECT *
FROM gh.pypi_meta_slug_cleaner_04022022
where slug is not null
'''
df_slug_commits = load_db_table(query, engine_p)
df_slug_commits

In [ ]:
missing_slugs_in_cost = df_slug_commits_cost[df_slug_commits_cost.slug.isna()==True].drop_duplicates().slug_meta.to_list()

In [ ]:
missing_slugs_in_cost_name = df_slug_commits[df_slug_commits.slug.isin(missing_slugs_in_cost)].name.to_list()

In [ ]:
len(missing_slugs_in_cost_name)

In [ ]:
#testing for number of packages which were created afte 2019 on pypi
new_packages = pd.read_csv('pre_2020_gh_package.csv')
new_packages.name = new_packages.name.apply(lambda x: str(x).lower())
new_packages

In [ ]:
new_packages[new_packages.name.isin(missing_slugs_in_cost_name)]

In [ ]:
df_slug_commits[df_slug_commits.name.isin(new_packages.name.to_list())]

In [ ]:
#query to save all slug names from sponsors github repo whic hdid nto join with slug_meta
query_cost_join = '''
select slug_meta from gh.pypi_cost_by_repo_clean_lower_04022022
where slug is null

'''
slug_didnot_join = load_db_table(query_cost_join, engine_p)

In [ ]:
slug_didnot_join.to_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/slugs_did_not_join_with_commits.csv')

## Create a materialized view for repos and costs

In [ ]:
# inner join on commits data 
query_cost_join = '''
CREATE MATERIALIZED VIEW gh.pypi_cost_by_repo_clean_lower_04022022_2 
    AS (SELECT gh.pypi_meta_slug_cleaner_04022022.slug AS slug_meta, 
    gh.cost_by_repo_0919_dd_nmrc_jbsc_.* 
    FROM gh.pypi_meta_slug_cleaner_04022022 
    INNER JOIN gh.cost_by_repo_0919_dd_nmrc_jbsc_ 
    ON 
    LOWER(gh.pypi_meta_slug_cleaner_04022022.slug)=LOWER(gh.cost_by_repo_0919_dd_nmrc_jbsc_.slug));

'''
load_db_table(query_cost_join, engine_p)

In [ ]:
query = '''
SELECT *
FROM gh.pypi_cost_by_repo_clean_lower_04022022_2
'''
df_slug_commits_cost = load_db_table(query, engine_p)
df_slug_commits_cost

## Other sql queries used to create views for reference

In [ ]:
#this was converted to a db table pypi_cost_by_repo_clean_lower
# gh.commits_dd_nmrc_jbsc_

# command used to make table in terminal shell
'''
CREATE MATERIALIZED VIEW gh.pypi_cost_by_repo_clean_lower_ 
AS (SELECT gh.pypi_meta_slug_clean.slug AS slug_meta, gh.cost_by_repo_0919_dd_nmrc_jbsc.* 
FROM gh.pypi_meta_slug_clean 
LEFT JOIN gh.cost_by_repo_0919_dd_nmrc_jbsc 
ON LOWER(gh.pypi_meta_slug_clean.slug)=LOWER(gh.cost_by_repo_0919_dd_nmrc_jbsc.slug));'''

''' 

CREATE MATERIALIZED VIEW gh.pypi_cost_by_repo_clean_lower_ 
        AS (SELECT gh.pypi_meta_slug_clean.slug AS slug_meta, gh.cost_by_repo_0919_dd_nmrc_jbsc_.* 
        FROM gh.pypi_meta_slug_clean 
        LEFT JOIN gh.cost_by_repo_0919_dd_nmrc_jbsc_ 
        ON LOWER(gh.pypi_meta_slug_clean.slug)=LOWER(gh.cost_by_repo_0919_dd_nmrc_jbsc_.slug));
'''


In [ ]:
'''
CREATE MATERIALIZED VIEW gh.pypi_cost_by_repo_clean_lower 
AS (SELECT gh.pypi_meta_slug_clean.slug AS slug_meta, gh.commits_dd_nmrc_jbsc_.* 
FROM gh.pypi_meta_slug_clean 
LEFT JOIN gh.cost_by_repo_0919_dd_nmrc_jbsc 
ON LOWER(gh.pypi_meta_slug_clean.slug)=LOWER(gh.cost_by_repo_0919_dd_nmrc_jbsc.slug));
'''

## Table cost_by_repo_0919_dd_nmrc_jbsc Creation
* created using oss_2021_pypi_gh_cost_by_repo.sql

In [ ]:
'''
CREATE MATERIALIZED VIEW gh.cost_by_repo_0919_dd_nmrc_jbsc AS (
WITH commits_annual AS (
SELECT slug, login, additions, deletions, EXTRACT(YEAR FROM committed_date)::int AS year
FROM gh.commits_dd_nmrc_jbsc_
)

SELECT slug, COUNT(*) AS commits, SUM(additions) AS additions, SUM(deletions) AS deletions,
					SUM(additions + deletions) AS sum_adds_dels, SUM(additions - deletions) AS net_adds_dels
FROM commits_annual
WHERE year > 2008 AND year < 2020
GROUP BY slug
);
--old table 7622930
--new table 7840227
GRANT ALL PRIVILEGES ON TABLE gh.cost_by_repo_0919_dd_nmrc_jbsc TO ncses_oss;
'''

## Table for top contributors of a repo
* Getting max commits for country selection

In [ ]:
myquery = """

SELECT a.*, b.country 
FROM(     
    SELECT m.slug, m.login, MAX(total_additions + total_deletions) as total_commits
    FROM (
        SELECT p.slug, c.login,
            sum(c.additions) OVER(PARTITION BY p.slug, c.login) AS total_additions,
            sum(c.deletions) OVER(PARTITION BY p.slug, c.login) AS total_deletions
        FROM gh.pypi_cost_by_repo_clean_lower_04022022_2 as p
        INNER JOIN gh.commits_dd_nmrc_jbsc_ as c ON p.slug=c.slug
    ) AS m
    GROUP BY m.slug, m.login
    ORDER BY slug, total_commits DESC
) AS a
INNER JOIN gh_cost.sectored_fractioned_103121 as b ON a.login = b.login

"""
pkg_cost_tbl = load_db_table(myquery,engine_p)

# have yet to figure out how to filter to only the login with max commits but can do it in python 
pkg_cost_tbl = pkg_cost_tbl.dropna()
pkg_cost_tbl_max = pkg_cost_tbl.groupby(['slug'])['total_commits'].max()
pkg_cost_tbl_max = pd.DataFrame(pkg_cost_tbl_max).reset_index().merge(pkg_cost_tbl, on=['slug', 'total_commits'])
pkg_cost_tbl_max.to_csv('max_contrib_country_per_package.csv', index=False)

## Other notebook references
* temp_meta_1 -> temp_meta_creation.ipynb
* Dependency Edgelist - oss_2021_pypi_dependency_edgelist
* Contributor network edge list- oss_2021_pypi_contributors_join_11202021.sql 
* Cleaning slugs - oss_pypi_2021_meta_cleaning_slugs_02122022.ipynb
* cost_by_repo_0919_dd_nmrc_jbsc - oss_2021_pypi_gh_cost_by_repo.sql
* downloads - oss_2021_meta_download_EDA
* sector stuff - oss_2021_sector_download_EDA.ipynb